# LightGBM

In [28]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy.random import seed

# 随机种子
seed(4)

# 读取CSV文件，指定"gene"列作为索引
label = pd.read_csv('~/wuzhenao/divide/Seqscope_Find_var.csv', index_col='gene')['label']

# 读取数据，排除"gene"和"label"列
data = pd.read_csv('~/wuzhenao/divide/Seqscope_Find_var.csv').drop(columns=['gene', 'label'])

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.25)

# 转换为Dataset数据格式
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

# 参数
params = {
    'learning_rate': 0.2,
    'lambda_l1': 0.4,
    'lambda_l2': 0.4,
    'max_depth': 8,
    'objective': 'multiclass',  # 目标函数
    'num_class': 2,
}

# 模型训练
gbm = lgb.train(params, train_data, valid_sets=[validation_data])

# 模型预测
y_pred = gbm.predict(X_test)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print(accuracy_score(y_test, y_pred))


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] There are no meaningful features, as all feature values are constant.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -0.179924
[LightGBM] [Info] Start training from score -1.803832
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[1]	valid_0's multi_logloss: 0.674695
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[2]	valid_0's multi_logloss: 0.674695
[Ligh

# Random Forest

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy.random import seed

# 读取CSV文件，指定"gene"列作为索引
data = pd.read_csv('~/wuzhenao/divide/Seqscope_Find_var.csv')

# 使用LabelEncoder对基因列进行编码
gene_encoder = LabelEncoder()
data['gene'] = gene_encoder.fit_transform(data['gene'])

# 将标签列的数据类型转换为整数
data['label'] = data['label'].astype(int)

# 划分训练集和测试集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data.drop(columns=['label']), data['label'], test_size=0.25, random_state=1)

# 将Pandas DataFrame转换为NumPy数组
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)
Xtest = np.array(Xtest)
Ytest = np.array(Ytest)

# 建立模型
clf = DecisionTreeClassifier(random_state=1, max_depth=5)  # 限制决策树的深度
rfc = RandomForestClassifier(random_state=1, n_estimators=100)  # 增加随机森林的树的数量

clf = clf.fit(Xtrain, Ytrain)
rfc = rfc.fit(Xtrain, Ytrain)

# 查看模型效果
score_c = clf.score(Xtest, Ytest)
score_r = rfc.score(Xtest, Ytest)

# 打印最后结果
print("Single Tree:", score_c)
print("Random Forest:", score_r)


Single Tree: 0.798
Random Forest: 0.784


# CPPLS-MLP


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 读取CSV文件，指定"gene"列作为索引
data = pd.read_csv('~/wuzhenao/divide/Seqscope_Find_var.csv')

# 将基因名称进行独热编码
gene_encoder = OneHotEncoder(sparse=False)
gene_encoded = gene_encoder.fit_transform(data[['gene']])

# 读取标签列
label = data['label']

# 划分训练集和测试集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(gene_encoded, label, test_size=0.25, random_state=2)

# 特征标准化
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

# 建立神经网络模型
model = keras.Sequential([
    layers.Input(shape=(Xtrain.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(Xtrain, Ytrain, epochs=100, batch_size=32, validation_data=(Xtest, Ytest))

# 评估模型
test_loss, test_acc = model.evaluate(Xtest, Ytest)
print("Test accuracy:", test_acc)


/home/tjzhang01/miniconda3/envs/PLS/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100
47/47 [==============================] - 1s 7ms/step - loss: 0.5038 - accuracy: 0.8020 - val_loss: 0.5543 - val_accuracy: 0.8300
Epoch 2/100
47/47 [==============================] - 0s 3ms/step - loss: 0.2508 - accuracy: 0.8953 - val_loss: 0.4917 - val_accuracy: 0.8300
Epoch 3/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0734 - accuracy: 0.9907 - val_loss: 0.4585 - val_accuracy: 0.8300
Epoch 4/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.4576 - val_accuracy: 0.8300
Epoch 5/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.4618 - val_accuracy: 0.8300
Epoch 6/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.4668 - val_accuracy: 0.8300
Epoch 7/100
47/47 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.4721 - val_accuracy: 0.8300
Epoch 8/100
4